# Insight to Impact: Feeling Through Another Way
**Hands‑on B (20 minutes)**

This module shows how a vision-language model (VLM) can narrate an image in different accessibility tones: elder-friendly, cognitive-friendly, or guide-style, and answer region-specific questions.

1) Upload or capture an image.  
2) Choose a style and optional region crop.  
3) Model narrates the scene and answers questions.  
4) (Optional) TTS reads the narration aloud.

**Model**: Qwen2-VL-2B-Instruct

**Core features:** scene narration, region Q&A, style control, TTS, export text  

## **GPU Version**


## 0. Runtime setup

In [1]:
# Use GPU runtime in Colab
!pip -q install transformers accelerate timm torch torchvision gradio gtts pillow

import torch, cv2, json, io
import numpy as np
from PIL import Image
import gradio as gr
from gtts import gTTS
from transformers import AutoProcessor, AutoModelForVision2Seq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.1 MB/s eta 0:00:00


## 1. Download Model (Qwen2-VL-2B Instruct quantized)

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "Qwen/Qwen2-VL-2B-Instruct"
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True
).eval()

print("✓ Qwen2-VL-2B-Instruct loaded on", device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

✓ Qwen2-VL-2B-Instruct loaded on cpu


## 2. Prompt Templates for Different Styles

In [3]:
STYLE_PROMPTS = {
    "normal": "Describe this image in detail.",
    "elder": "Narrate for elder users using short simple sentences and gentle tone.",
    "cognitive": "Explain what is happening step by step in simple commands.",
    "guide": "Guide the listener: start with overall view, then describe key items and possible actions."
}

## 3. Core Functions

In [4]:
def generate_caption(image_pil, style="normal", region=None):
    img = image_pil if region is None else image_pil.crop(region)
    task_prompt = STYLE_PROMPTS.get(style, STYLE_PROMPTS["normal"])
    query = f"<image>\n{task_prompt}\nPlease summarize into three bullet points."

    inputs = processor(images=img, text=query, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        out = model.generate(**inputs, max_new_tokens=256)
    text = processor.batch_decode(out, skip_special_tokens=True)[0]
    return text.strip()

def tts_save(text, fname="narration.mp3", speed=1.0):
    try:
        tts = gTTS(text=text, lang="en", slow=(speed<1.0))
        tts.save(fname)
        return fname
    except Exception as e:
        print("TTS failed:", e)
        return None


## 4. Gradio Interface

In [6]:
def run_vlm(image, style, region_x, region_y, region_w, region_h, tts, tts_speed):
    if image is None:
        return None, None, "{}"
    img = Image.fromarray(image.astype(np.uint8))
    region = None
    if all(v is not None for v in [region_x, region_y, region_w, region_h]) and region_w>0 and region_h>0:
        region = (region_x, region_y, region_x+region_w, region_y+region_h)
    text = generate_caption(img, style, region)
    audio_path = tts_save(text, "narration.mp3", tts_speed) if tts else None
    with open("alt_text.txt","w") as f: f.write(text)
    return text, audio_path, json.dumps({"style":style, "region":region}, indent=2)

with gr.Blocks(title="Visual Narration & Task Coach") as demo:
    gr.Markdown("### Upload → Select Style → Narrate → (Optionally Listen)")
    with gr.Row():
        with gr.Column():
            img = gr.Image(label="Upload or Camera", sources=["upload","webcam"], type="numpy")
            style = gr.Radio(["normal","elder","cognitive","guide"], value="elder", label="Narration Style")
            with gr.Accordion("Optional Region Crop"):
                region_x = gr.Number(label="x", value=None)
                region_y = gr.Number(label="y", value=None)
                region_w = gr.Number(label="width", value=None)
                region_h = gr.Number(label="height", value=None)
            tts = gr.Checkbox(label="Enable TTS Audio", value=True)
            tts_speed = gr.Slider(0.5,1.0,step=0.25,value=1.0,label="TTS Speed")
            run_btn = gr.Button("Run Narration")
        with gr.Column():
            out_text = gr.Textbox(label="Narration Output", lines=10)
            out_audio = gr.Audio(label="Audio Playback", type="filepath")
            out_json = gr.JSON(label="Meta Info / Alt-Text Export")
    run_btn.click(run_vlm,
                  inputs=[img,style,region_x,region_y,region_w,region_h,tts,tts_speed],
                  outputs=[out_text,out_audio,out_json])

# To start locally (faster than share=True)
demo.launch(share=False)
print("UI ready — default style: elder friendly.")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

UI ready — default style: elder friendly.


## 5. Mini‑Exercises


1. **Try all styles.**  
   Notice how sentence length and tone change across elder, cognitive, and guide modes.

2. **Region Q&A.**  
   Enter a region and ask “What is this region?”  
   Observe how description changes with context.

3. **Modify the prompt.**  
   Add emotional tone:  
   `"Describe warmly and encouragingly for older adults."`

4. **Export alt-text.**  
   File `alt_text.txt` can serve as accessible metadata for web images.


## **CPU Version**

## 0. Runtime setup

In [6]:
%pip -q install transformers pillow torch torchvision gradio gTTS paddlepaddle paddleocr

In [8]:
%pip install -U "gradio>=4.31"

## 1. Core Functions

In [16]:
import torch, numpy as np, json
from PIL import Image
import gradio as gr
from transformers import BlipProcessor, BlipForConditionalGeneration
from gtts import gTTS

# load BLIP captioner
blip_name = "Salesforce/blip-image-captioning-base"
blip_processor = BlipProcessor.from_pretrained(blip_name)
blip_model = BlipForConditionalGeneration.from_pretrained(blip_name)
blip_model.eval()

# optional OCR
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

STYLE_PROMPTS = {
    "normal": "Describe this image in detail.",
    "elder":  "Use short, gentle sentences with everyday words.",
    "cognitive": "Explain step by step in simple commands.",
    "guide":  "Start with the overall view, then key items, then suggested actions."
}

def blip_caption(pil_img, max_new_tokens=40):
    inputs = blip_processor(pil_img, return_tensors="pt")
    with torch.inference_mode():
        out_ids = blip_model.generate(**inputs, max_new_tokens=max_new_tokens, num_beams=1)
    return blip_processor.decode(out_ids[0], skip_special_tokens=True).strip()

def run_ocr(pil_img):
    import numpy as np
    arr = np.array(pil_img)[:, :, ::-1]
    try:
        res = ocr.ocr(arr)
    except TypeError:
        res = ocr.ocr(arr, cls=True)

    lines = []
    for page in res:
        for line in page:
            txt = line[1][0] if isinstance(line, (list, tuple)) and len(line) > 1 else ""
            if txt and txt.strip():
                lines.append(txt.strip())
    return " ".join(lines[:20])

def style_rewrite(caption, ocr_text, style):
    bullets = []
    if style == "elder":
        bullets = [
            f"{caption}.",
            ("I can read some text: " + ocr_text) if ocr_text else "No clear text found.",
            "You can ask me to focus on a smaller area."
        ]
        return " ".join(bullets)
    if style == "cognitive":
        steps = []
        steps.append("Step 1: Look at the main objects.")
        steps.append(f"Step 2: Summary — {caption}.")
        if ocr_text:
            steps.append(f"Step 3: Read the text: {ocr_text}.")
        steps.append("Step 4: Ask follow-up questions about a region.")
        return "\n".join(steps)
    if style == "guide":
        s = []
        s.append("Overview: " + caption + ".")
        if ocr_text:
            s.append("Key text: " + ocr_text + ".")
        s.append("Action: Zoom or click an area to learn more.")
        return " ".join(s)
    # normal
    return caption if not ocr_text else f"{caption}. Text: {ocr_text}"

def tts_save(text, fname="narration.mp3", speed=1.0):
    try:
        gTTS(text=text, lang="en", slow=(speed<1.0)).save(fname); return fname
    except Exception as e:
        print("TTS failed:", e); return None

def narrate(image_np, style, region):  # region: (x,y,w,h) or None
    img = Image.fromarray(image_np.astype(np.uint8))
    roi = img if region is None else img.crop((region[0], region[1], region[0]+region[2], region[1]+region[3]))

    cap = blip_caption(roi, max_new_tokens=40)
    ocr_txt = run_ocr(roi)
    out_text = style_rewrite(cap, ocr_txt, style)
    return out_text

/tmp/ipython-input-3180650553.py:15: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  ocr = PaddleOCR(use_angle_cls=True, lang='en')
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_m

## 2. Gradio Interface

In [19]:
import gradio as gr
import numpy as np
from PIL import Image
import json

def run_ui_simple(full_img, style, use_region, x, y, w, h, tts, tts_speed):
    if full_img is None:
        return "", None, "{}"

    src_np = full_img
    region = None

    # optional numeric region (stable on any gradio version)
    if use_region and all(v is not None for v in [x, y, w, h]) and w > 0 and h > 0:
        x, y, w, h = int(x), int(y), int(w), int(h)
        img = Image.fromarray(full_img.astype(np.uint8))
        crop = img.crop((x, y, x + w, y + h))
        src_np = np.array(crop)
        region = (x, y, w, h)

    # narrate (your BLIP+OCR pipeline)
    text = narrate(src_np, style, region=None)  # region already applied above if any

    # TTS
    audio_path = tts_save(text, fname="narration.mp3", speed=tts_speed) if tts else None

    # exports
    with open("alt_text.txt", "w") as f:
        f.write(text)
    meta = {"style": style, "used_region": bool(region is not None), "region": region}

    return text, audio_path, json.dumps(meta, indent=2)

with gr.Blocks(title="CPU Visual Narration (BLIP + OCR, No Crop UI)") as demo:
    gr.Markdown("### Upload → Choose Style → Narrate → (Optional) TTS")

    with gr.Row():
        with gr.Column():
            img_full = gr.Image(label="Original Image", sources=["upload","webcam"], type="numpy")

            style = gr.Radio(
                ["normal", "elder", "cognitive", "guide"],
                value="elder",
                label="Narration Style"
            )

            # Optional numeric region (off by default)
            use_region = gr.Checkbox(value=False, label="Use region by coordinates (optional)")
            with gr.Row():
                region_x = gr.Number(label="x", value=None)
                region_y = gr.Number(label="y", value=None)
            with gr.Row():
                region_w = gr.Number(label="width", value=None)
                region_h = gr.Number(label="height", value=None)

            tts = gr.Checkbox(value=True, label="Enable TTS")
            tts_speed = gr.Slider(0.5, 1.0, value=1.0, step=0.25, label="TTS Speed")

            run_btn = gr.Button("Run Narration")
        with gr.Column():
            out_text = gr.Textbox(label="Narration Output", lines=12)
            out_audio = gr.Audio(label="Audio Playback", type="filepath")
            out_json  = gr.JSON(label="Meta")
            gr.Markdown("**Exports:** `alt_text.txt`, `narration.mp3`.")

    run_btn.click(
        fn=run_ui_simple,
        inputs=[img_full, style, use_region, region_x, region_y, region_w, region_h, tts, tts_speed],
        outputs=[out_text, out_audio, out_json]
    )

demo.launch(share=False, inbrowser=False, debug=True, show_error=True)
print("UI ready. Leave 'Use region by coordinates' unchecked to narrate the whole image.")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
UI ready. Leave 'Use region by coordinates' unchecked to narrate the whole image.
